In [1]:
# Table table-retrieval-effectiveness-over-time

In [20]:
#!pip install trectools

from trectools import TrecQrel, TrecRun, TrecEval

In [6]:
RUN_DIR_MARCO='/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-probst/retrievalExperiments/runs-ecir22/dl19+20/'

QRELS_MARCO_DL_19=TrecQrel('../../Data/dl19+20/2019qrels-docs.txt')
QRELS_MARCO_DL_20=TrecQrel('../../Data/dl19+20/2020qrels-docs.txt')

In [23]:
APPROACH_TO_MARCO_V1_RUN_FILE={
    'BM25@2016-07': 'run.cc-16-07-anchortext.bm25-default.txt',
    'BM25@2017-04': 'run.cc-17-04-anchortext.bm25-default.txt',
    'BM25@2018-13': 'run.cc-18-13-anchortext.bm25-default.txt',
    'BM25@2019-47': 'run.cc-19-47-anchortext.bm25-default.txt',
    'BM25@2020-05': 'run.cc-20-05-anchortext.bm25-default.txt',
    'BM25@2021-04': 'run.cc-21-04-anchortext.bm25-default.txt',
    'BM25@16--21': 'run.cc-combined-anchortext.bm25-default.txt',
    'BM25@Content': 'run.ms-marco-content.bm25-default.txt',
    'BM25@Orcas': 'run.orcas.bm25-default.txt',
    'DeepCT@Anchor': 'run.ms-marco-deepct-v1-anserini-docs-cc-2019-47-sampled-test-overlap-removed-389979.bm25-default.txt',
    'DeepCT@Orcas': 'run.ms-marco-deepct-v1-anserini-docs-orcas-sampled-test-overlap-removed-390009.bm25-default.txt',
    'DeepCT@Train':'run.ms-marco-deepct-v1-anserini-docs-ms-marco-training-set-test-overlap-removed-389973.bm25-default.txt',
    'MonoT5': 'run.ms-marco-content.bm25-mono-t5-maxp.txt',
    'MonoBERT': 'run.ms-marco-content.bm25-mono-bert-maxp.txt',
    'LambdaMART@CTA':'run.ms-marco.lambda-mart-cta-trees-1000.txt',
    'LambdaMART@CTOA':'run.ms-marco.lambda-mart-ctoa-trees-1000.txt',
    'LambdaMART@CTO':'run.ms-marco.lambda-mart-cto-trees-1000.txt',
    'LambdaMART@CT':'run.ms-marco.lambda-mart-ct-trees-1000.txt',
}


In [24]:
def is_anchor_text(run_file):
    return '-anchortext' in run_file

def format_score(score, run_file, position):
    # Manually maintained
    if is_anchor_text(run_file):
        max_scores = [0.40, 0.37, 0.33, 0.31]
    else:
        max_scores = [0.67, 0.63, 0.61, 0.62]
        
    ret = '{:.2f}'.format(score)
    if max_scores[position] <= score:
        return '{\\textbf{' + ret + '}} '
    else:
        return ret

def eval_on_marco_dl19(run_file):
    ret = ' '
    
    run = TrecRun(RUN_DIR_MARCO + '/' + run_file)
    run.run_data = run.run_data[run.run_data['query'].isin(QRELS_MARCO_DL_19.qrels_data['query'].unique())]
    trec_eval=TrecEval(run, QRELS_MARCO_DL_19)
    
    ret += '& ' + format_score(trec_eval.get_ndcg(depth=10, removeUnjudged=True), run_file, 0)
    ret += '& ' + format_score(trec_eval.get_ndcg(depth=20, removeUnjudged=True), run_file, 1)
    
    return ret + '&'

def eval_on_marco_dl20(run_file):
    ret = ' '
    
    run = TrecRun(RUN_DIR_MARCO + '/' + run_file)
    run.run_data = run.run_data[run.run_data['query'].isin(QRELS_MARCO_DL_20.qrels_data['query'].unique())]
    trec_eval=TrecEval(run, QRELS_MARCO_DL_20)
    
    ret += format_score(trec_eval.get_ndcg(depth=10, removeUnjudged=True), run_file, 2)
    ret += '& ' + format_score(trec_eval.get_ndcg(depth=20, removeUnjudged=True), run_file, 3)
    
    return ret

def table_row(approach):
    v1_eval = ' & --- & --- &'
    
    if approach in APPROACH_TO_MARCO_V1_RUN_FILE:
        v1_eval = eval_on_marco_dl19(APPROACH_TO_MARCO_V1_RUN_FILE[approach])
    
    v2_eval = ' --- & ---'
    
    if approach in APPROACH_TO_MARCO_V1_RUN_FILE:
        v2_eval = eval_on_marco_dl20(APPROACH_TO_MARCO_V1_RUN_FILE[approach])
    
    return '& ' + approach + v1_eval + v2_eval + ' \\\\'


def table_entry_page_retrieval_effectiveness():
    return '''\\begin{table*}[bt]
    \\setlength{\\tabcolsep}{0.18em}
    \\caption{TBD.}
    \\label{table-retrieval-effectiveness-over-time}
    \\scriptsize
    \\begin{minipage}{0.4\\textwidth}
    \\scriptsize
    (a) TBD.\\\\
    \\includegraphics[width=1\\textwidth]{figure-retrieval-effectiveness-over-time}
    \\end{minipage}%
    \\hfill
    \\begin{tabular*}{0.6\\textwidth}{@{\\extracolsep{\\fill}}ll@{\\quad}ccc@{\\quad}ccc@{}}
        \\multicolumn{3}{@{}l@{}}{(b) TBD.}\\\\
        \\toprule

        & & \\multicolumn{2}{@{}c@{\\quad}}{DL 2019} & \\multicolumn{2}{@{}c@{\\quad}}{DL 2020}\\\\

        \\cmidrule(r{1em}){3-4} \\cmidrule(r{1em}){5-6}

        & & nDCG@10 & nDCG@20 & nDCG@10 & nDCG@20\\\\

        \\midrule

        \\multirow{7}{*}{\\rotatebox[origin=c]{90}{\\parbox[c]{4em}{\\centering \\textbf{Anchor}}}}
        ''' + table_row('BM25@2016-07') + '''
        
        ''' + table_row('BM25@2017-04') + '''

        ''' + table_row('BM25@2018-13') + '''

        ''' + table_row('BM25@2019-47') + '''

        ''' + table_row('BM25@2020-05') + '''

        ''' + table_row('BM25@2021-04') + '''
        
        ''' + table_row('BM25@16--21') + '''

        \\midrule

        \\multirow{7}{*}{\\rotatebox[origin=c]{90}{\\parbox[c]{4em}{\\centering \\textbf{Baselines}}}}
        ''' + table_row('BM25@Content') + '''

        ''' + table_row('BM25@Orcas') + '''
        
        ''' + table_row('DeepCT@Anchor') + '''

        ''' + table_row('DeepCT@Orcas') + '''

        ''' + table_row('DeepCT@Train') + '''
        
        ''' + table_row('MonoT5') + '''
        
        ''' + table_row('MonoBERT') + '''

        ''' + table_row('LambdaMART@CTOA') + '''
        
        ''' + table_row('LambdaMART@CTO') + '''
        
        ''' + table_row('LambdaMART@CTA') + '''
        
        ''' + table_row('LambdaMART@CT') + '''

        \\bottomrule

    \\end{tabular*}
    \\vspace*{-2ex}
\\end{table*}
'''

print(table_entry_page_retrieval_effectiveness())

\begin{table*}[bt]
    \setlength{\tabcolsep}{0.18em}
    \caption{TBD.}
    \label{table-retrieval-effectiveness-over-time}
    \scriptsize
    \begin{minipage}{0.4\textwidth}
    \scriptsize
    (a) TBD.\\
    \includegraphics[width=1\textwidth]{figure-retrieval-effectiveness-over-time}
    \end{minipage}%
    \hfill
    \begin{tabular*}{0.6\textwidth}{@{\extracolsep{\fill}}ll@{\quad}ccc@{\quad}ccc@{}}
        \multicolumn{3}{@{}l@{}}{(b) TBD.}\\
        \toprule

        & & \multicolumn{2}{@{}c@{\quad}}{DL 2019} & \multicolumn{2}{@{}c@{\quad}}{DL 2020}\\

        \cmidrule(r{1em}){3-4} \cmidrule(r{1em}){5-6}

        & & nDCG@10 & nDCG@20 & nDCG@10 & nDCG@20\\

        \midrule

        \multirow{7}{*}{\rotatebox[origin=c]{90}{\parbox[c]{4em}{\centering \textbf{Anchor}}}}
        & BM25@2016-07 & 0.37& 0.32& 0.28& 0.24 \\
        
        & BM25@2017-04 & 0.39& 0.35& 0.29& 0.27 \\

        & BM25@2018-13 & 0.35& 0.33& 0.27& 0.25 \\

        & BM25@2019-47 & 0.34& 0.31& 0.27& 0.25 \